In [40]:
import pandas as pd

In [49]:
#data=pd.read_csv('C:/Users/danto/Desktop/r.csv')ml-100k/u.item
data=pd.read_csv("C:/Users/danto/Desktop/IMDB Dataset.csv",encoding='ISO-8859-1')
##pd.read_csv('', sep='|', names=m_cols , encoding='latin-1')

In [50]:
#First 10 results
data.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [52]:
data['review'][0:2]

0    One of the other reviewers has mentioned that ...
1    A wonderful little production. <br /><br />The...
Name: review, dtype: object

#### Next find vectors

In [53]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [54]:
count=CountVectorizer()

In [55]:
text = np.array(['Its raining outside',
                 'The perfect weather for having tea anf fritters',
                 'It would be nice to share it with friends, family or loved ones'])

In [56]:
collect=count.fit_transform(text)
print(count.vocabulary_)

{'its': 8, 'raining': 15, 'outside': 13, 'the': 18, 'perfect': 14, 'weather': 20, 'for': 3, 'having': 6, 'tea': 17, 'anf': 0, 'fritters': 5, 'it': 7, 'would': 22, 'be': 1, 'nice': 10, 'to': 19, 'share': 16, 'with': 21, 'friends': 4, 'family': 2, 'or': 12, 'loved': 9, 'ones': 11}


In [57]:
print(collect.toarray())

[[0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0]
 [1 0 0 1 0 1 1 0 0 0 0 0 0 0 1 0 0 1 1 0 1 0 0]
 [0 1 1 0 1 0 0 2 0 1 1 1 1 0 0 0 1 0 0 1 0 1 1]]


### Term Frequency and Inverse Document Frequency


In [58]:
from sklearn.feature_extraction.text import TfidfTransformer
tdf=TfidfTransformer(use_idf=True,norm='l2',smooth_idf=True)

In [63]:
#tf-idf (t,d) = tf(t,d) X idf(t,d)
#idf(t,d) = log(nd / 1 + df(d,t))
#nd = total no docs and df(d,t) = no of docs that contain the term t
print(tdf.fit_transform(count.fit_transform(text)).toarray())


[[0.   0.   0.   0.   0.   0.   0.   0.   0.58 0.   0.   0.   0.   0.58
  0.   0.58 0.   0.   0.   0.   0.   0.   0.  ]
 [0.35 0.   0.   0.35 0.   0.35 0.35 0.   0.   0.   0.   0.   0.   0.
  0.35 0.   0.   0.35 0.35 0.   0.35 0.   0.  ]
 [0.   0.26 0.26 0.   0.26 0.   0.   0.52 0.   0.26 0.26 0.26 0.26 0.
  0.   0.   0.26 0.   0.   0.26 0.   0.26 0.26]]


In [64]:
np.set_printoptions(precision=2)
print(tdf.fit_transform(count.fit_transform(text)).toarray())

[[0.   0.   0.   0.   0.   0.   0.   0.   0.58 0.   0.   0.   0.   0.58
  0.   0.58 0.   0.   0.   0.   0.   0.   0.  ]
 [0.35 0.   0.   0.35 0.   0.35 0.35 0.   0.   0.   0.   0.   0.   0.
  0.35 0.   0.   0.35 0.35 0.   0.35 0.   0.  ]
 [0.   0.26 0.26 0.   0.26 0.   0.   0.52 0.   0.26 0.26 0.26 0.26 0.
  0.   0.   0.26 0.   0.   0.26 0.   0.26 0.26]]


### Tokenization

In [65]:
from nltk.stem.porter import PorterStemmer
port=PorterStemmer()

In [66]:
def stemmer_tokenize(text):
    return [port.stem(word) for word in text.split()]
stemmer_tokenize('You have to do what you have got to do')

['you', 'have', 'to', 'do', 'what', 'you', 'have', 'got', 'to', 'do']

### Vectorization

In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [68]:
tdf=TfidfVectorizer(strip_accents=None,lowercase=False,tokenizer=stemmer_tokenize,use_idf=True,norm='l2',smooth_idf=True)

In [70]:
y=data.sentiment.values
x=tdf.fit_transform(data.review)

In [71]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=1,test_size=0.4, shuffle='False')

In [72]:
import pickle
from sklearn.linear_model import LogisticRegressionCV

In [85]:
c=LogisticRegressionCV(cv=5,scoring='accuracy',random_state=32,n_jobs=3,verbose=1,max_iter=200).fit(x_train,y_train)
saved_model=open('saved_model.sav','wb')

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed:  4.3min finished


In [86]:
pickle.dump(c,saved_model)
saved_model.close()

In [87]:
filename='saved_model.sav'
saved_c=pickle.load(open(filename,'rb'))


In [88]:
saved_c.score(x_test,y_test)

C:\Users\danto\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:2260: ChangedBehaviorWarning: The long-standing behavior to use the accuracy score has changed. The scoring parameter is now used. This warning will disappear in version 0.22.
  ChangedBehaviorWarning)


0.8938